In [2]:
!pip install labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl (38.1 MB)
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  Running setup.py install for tensorflow-gpu: started
  Running setup.py install for tensorflow-gpu: finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\GÃºssem\AppData\Local\Temp\pip-install-n5bgl_sx\tensorflow-gpu_9c43ae31c90d40a79f92aa3d775f86d9\setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subprocess

# 1. Obtenció i tractament de dades

### 1.1 Ús de la llibreria LabelMe

* Executem la llibreria LabelMe, que manualment ens permetrà obtenir unes coordenades que marquin la àrea del objecte que volem detectar. Aquesta llibreria ens retornarà un arxiu .json amb els valors numèrics dels punts marcats. Amb aquests valors, la xarxa neuronal serà capaç d'entrenar-se. Haurem de crear una carpeta que anomenarem "labels" on hi aniran aquestes "etiquetes" en format .json

In [3]:
!labelme 

###  1.2 Creació de la base de dades

In [3]:
#importació de llibreries necessàries
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
import os
import random
import shutil

In [ ]:
#limitació de la memòria GPU 
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

### Carregar les imatges a TF 

In [ ]:
imatges = tf.data.Dataset.list_files('data\\images\\*.png') # introduïm les imatges a la base de dades

In [ ]:
def carregar_imatge(imatge): #funció per a carregar imatges
    byte_img = tf.io.read_file(imatge)
    img = tf.io.decode_png(byte_img)
    return img

In [ ]:
imatges = imatges.map(carregar_imatge) # executem la funció per carregar les imatges en la base de dades 

In [ ]:
imatges.as_numpy_iterator().next() 

In [ ]:
def imatgesDividir(dir_carpeta, n_imatges):
    num_elements = len(os.listdir(dir_carpeta))
    n_train = round(num_elements*0.7) # 70% de les imatges per entrenar
    n_test = round(num_elements*0.15) # 15% de les imatges per provar
    n_val = num_elements - (n_train + n_test) # 15% de les imatges per validar

    llista = []
    for i in range(1, (n_imatges + 1)):
        llista.append(i)

    if len(os.listdir(os.path.join(dir_carpeta,"test"))) < n_test:
        for i in range(n_test):
            n = random.choice(llista)
            nomt = str(str(n) + ".jpg")
            shutil.move(nomt, os.path.join(dir_carpeta,"test"))
            llista.remove(n)

    if len(os.listdir(os.path.join(dir_carpeta,"val"))) < n_val:
        for i in range(n_val):
            n = random.choice(llista)
            nomv = str(str(n) + ".jpg")
            shutil.move(nomv, os.path.join(dir_carpeta,"val"))
            llista.remove(n)

    if len(os.listdir(os.path.join(dir_carpeta,"val"))) >= n_val and len(os.listdir(os.path.join(dir_carpeta,"test"))) >= n_test: 
        for i in llista:
            nom = str(i) + ".jpg"
            shutil.move(os.path.join(dir_carpeta, nom), os.path.join(dir_carpeta, "train"))

In [ ]:
# Crear la carpeta principal
os.mkdir("dades")

# Crear les subcarpetes dins de la carpeta principal
for subcarpeta in ["train", "test", "val"]:
    path_subcarpeta = os.path.join("dades", subcarpeta)
    os.mkdir(path_subcarpeta)

imatgesDividir('T:\REP\deteccio_prova\data\images',300) #divisió de les carpetes i de les imatges

### Partició de les dades

In [ ]:
# mou les els labels a les seves respectives carpetes
for carpeta in ['train','test','val']:
    for arxiu in os.listdir(os.path.join('dades', carpeta, 'imagtes')): #per cada arxiu en cada carpeta
        nom_arxiu = arxiu.split('.')[0]+'.json' #extreu el nom de l'arxiu
        dir_existent = os.path.join('dades','labels', nom_arxiu)#crea possible direcció
        if os.path.exists(dir_existent): #si la possible direcció existeix, desplaça l'arxiu .json corresponent
            nova_dir = os.path.join('dades',carpeta,'labels',nom_arxiu)
            os.replace(dir_existent, nova_dir)#canviem

### 1.2.1 Ús de la llibreria Albumentation

In [4]:
import albumentations as alb
import cv2

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=1024, height=1024),
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='albumentations', #format de coords que volem (buscar docs)
                                                  label_fields=['class_labels']))

In [ ]:
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join('dades', partition, 'imagtes')):
        img = cv2.imread(os.path.join('dades', partition, 'imagtes', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('dades', partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [1024,1024,1024,1024]))
            # ^^^ carregar imatges i json ^^^

        try: 
            for x in range(60):# quantitat d'imatges que surten d'una imatge base
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['Maduixa'])
                cv2.imwrite(os.path.join('aug_data', partition, 'imagtes', f'{image.split(".")[0]}.{x}.jpg'),augmented['image'])
                
                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 
                    '''
                     La variable “annotation” és un diccionari que conté informació sobre la imatge i 
                     la seva anotació. Si el fitxer d'anotació existeix, el codi afegeix la caixa delimitadora
                     i la classe corresponent a l'objecte detectat a l'estructura “annotation”. 
                     Si el fitxer d'anotació no existeix, el codi assigna una caixa delimitadora i 
                     una classe buida a l'estructura “annotation”.
                    '''
                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

### Incloure imatges creades amb Albumentations a la Dataset

In [ ]:
#Obrim les carpetes amb les img a manipular i les carreguem
train_images = tf.data.Dataset.list_files('aug_data\\train\\images\\*.jpg',shuffle=False)
train_images = train_images.map(carregar_imatge)
#Es fa resize de les img i també baixem l'escala de la imatge a 1
train_images = train_images.map(lambda x: tf.image.resize(x, (250,250)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files('aug_data\\test\\images\\*.jpg', shuffle=False)
test_images = test_images.map(carregar_imatge)
test_images = test_images.map(lambda x: tf.image.resize(x, (250,250)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files('aug_data\\val\\images\\*.jpg', shuffle=False)
val_images = val_images.map(carregar_imatge)
val_images = val_images.map(lambda x: tf.image.resize(x, (250,250)))
val_images = val_images.map(lambda x: x/255)

### 2.5 Carregar etiquetes

In [ ]:
#funció que carrega els labels
def carregar_labels(x):
    with open(x.numpy(), 'r', encoding='utf-8') as f:
        label = json.load(f)
    return [label['class']],label['bbox']

In [ ]:
train_labels = tf.data.Dataset.list_files('aug_data\\train\\labels\\*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))#tf_py_function(funció, paràmetres, tipus de retorn) 
#obre els labels i els aplica la funció carregar_labels

In [ ]:
test_labels = tf.data.Dataset.list_files('aug_data\\test\\labels\\*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files('aug_data\\val\\labels\\*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))

### Combinar etiquetes i imatges

In [ ]:
#cal comprovar quantes imatges hi ha en cada carpeta
print("Train: " + str(len(train_labels)) + ", Test: " + str(len(test_labels)) + ", Val: " + str(len(val_labels)))

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels)) #mètode que junta les imatges amb els labels
train = train.shuffle(2000) #posar el número que hi ha apoximat cap amunt, en aquest cas 2000
train = train.batch(8)#crear lots de 8
train = train.prefetch(4)#redueix la capacitat de procesar per evitar errors

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(500)
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(500)
val = val.batch(8)
val = val.prefetch(4)

In [ ]:
train.as_numpy_iterator().next()[1]
'''
Amb això visualitzem la manera en què hem codificat i guardat les imatges;
Obtindrem això:
(array([[n],
        [n],
        [n],
        [n],
        [n],
        [n],
        [n],
        [n]], dtype=uint8),
 array([[n , n , n , n],
        [n , n , n , n],
        [n , n , n , n],
        [n , n , n , n],
        [n , n , n , n],
        [n , n , n , n],
        [n , n , n , n],
        [n , n , n , n]], dtype=float16))

la primera columna ens indica la classe de la imatge, és a dir, l'objecte que hi ha a la imatge
la segona columna ens indica la bounding box de la imatge, és a dir, la posició de l'objecte a la imatge

'''

# 2. Preparació de la intel·ligència artificial

In [ ]:
#La línia 2 importa les capes de Input, de convolució 2D, de connexió i de agregació màxima global
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

### 2.1 Descarregar i carregar el model VGG16

In [ ]:
vgg = VGG16(include_top=False) 
#Marca que les ultimes capes de la xarxa no les utilitzarem perquè afegirem les nostres pròpies

In [ ]:

'''
La funció crea i retorna un model de xarxa neuronal convolucional per a la detecció dels objectes. 
El model té dues parts: una per a determinar si una imatge conté algun objecte i una altra per a localitzar la maduixa en la imatge. 
Al utilitzar VGG16, com que esta pre-entrenada, les característiques del input
i les classifica en dues branques que son les dues parts mencionades abans de manera automàtica. 
'''
def build_model(): 
    input_layer = Input(shape=(250,250,3))
    
    vgg = VGG16(include_top=False)(input_layer)
    #Model de classificació
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1) #relu == funció que determina la classe; 
    class2 = Dense(1, activation='sigmoid')(class1) # sigmoid == funció que determina la presició de la classe

    # sigmoid = f(x) = 1/(1+e^-x)
    
    #Model de localització de coordenades
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)
    
    detector = Model(inputs=input_layer, outputs=[class2, regress2])
    return detector

In [ ]:
train.as_numpy_iterator().next()[1]

### 3.3 Prova de la xarxa neuronal

In [ ]:
detector = build_model()

In [ ]:
detector.summary()
'''
Aquest fragment dona una descripció resumida de la xarxa que acabem de construir,
inclou el nombre de capes, el nombre de paràmetres que ha après durant un entrenament i 
la forma de les dades que passen entre les capes.
'''

In [ ]:
X, y = train.as_numpy_iterator().next()

In [ ]:
X.shape

In [ ]:
classes, coords = detector.predict(X)

In [ ]:
classes, coords

### 2.2 Funcions de pèrdua i optimitzadors
Els optimitzadors són algorismes que s’utilitzen per ajustar els pesos d’una xarxa neuronal durant l’entrenament. Els optimitzadors són responsables de minimitzar la funció de pèrdua de la xarxa neuronal.

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001) #li introduïm el decay que hem calculat a l'optimitzador

'''
En el context de les xarxes neuronals, un optimitzador és un algorisme
que ajuda a ajustar els paràmetres de la xarxa per aconseguir una millor precisió.
L'optimitzador Adam és un exemple d'això i ajuda a l'optimitzador a convergir més ràpidament
i amb més precisió.
'''

### Creació de 'Localitzation Loss' i 'Classification Loss'

La funció té dos components: la pèrdua de localització i la pèrdua de classificació.
La pèrdua de localització mesura la diferència entre les coordenades dels quadres delimitadors 
predits i les coordenades dels quadres delimitadors reals. La pèrdua de classificació mesura la
diferència entre les probabilitats de classe predites i les probabilitats de classe reals. 
En aquesta funció, només es calcula la pèrdua de localització.

La funció té com a entrada dos tensors: y_true i yhat. 
y_true conté les coordenades dels quadres delimitadors reals i
les probabilitats de classe reals per a cada objecte en la imatge d’entrada.
yhat conté les coordenades dels quadres delimitadors predits i les
probabilitats de classe predites per a cada objecte en la imatge d’entrada.

La funció calcula la pèrdua de localització sumant el quadrat de la diferència
entre les coordenades dels quadres delimitadors reals i les coordenades dels quadres
delimitadors predits. A continuació, calcula la diferència entre l’amplada i l’alçada
dels quadres delimitadors reals i els quadres delimitadors predits i suma els quadrats
d’aquestes diferències. Finalment, retorna la suma de les dues pèrdues.

In [ ]:
def localization_loss(y_true, yhat):#primer valor: coordenades reals, segon valor: coordenades previstes     
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2])) #diferència dels dos primers valors de cada fila de la matriu
                  
    h_true = y_true[:,3] - y_true[:,1] #quarta columna d'una matriu - segona columna
    w_true = y_true[:,2] - y_true[:,0] #tercera columna - primera

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    '''
    delta_size = suma dels quadrats de les diferències entre les dimensions originals 
    i les dimensions reconstruïdes de l'imatge.
    '''
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    return delta_coord + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy() #model que fa una classificació binaria 
regressloss = localization_loss #model que acabem de crear

### Test de les mètriques de 'loss'

In [ ]:
#test, en les 2 cel·les inferiors s'hauria d'obtenir : <tf.Tensor: shape=(), dtype=float32, numpy=n>
regressloss(y[1], coords) #y[0] ==> 0 o 1 (hi ha maduixa o no) , y[1] ==> coords (tot del batch sencer)

In [ ]:
classloss(y[0], classes) #numpy ==> probabilitat

In [ ]:
localization_loss(y[1],coords)

# 3. Entrenament de la intel·ligència artificial

### 3.1 Creació del propi model 

In [ ]:
class Detector(Model): 
    def __init__(self, eyetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    '''
    La funció train_step executa cada pas de l'entrenament del model. El model reb un lot de data 
    del entrenament (data que hem escollit per a 'train'),i calcula la pèrdua total (total_loss) que 
    consisteix en la suma de la pèrdua de localització (batch_localizationloss) i la meitat de la
    pèrdua de classificació (batch_classloss). Finalment, calcula la perdua total i actualitza els
    pesos del optimitzador.
    '''
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            classes, coords = self.model(X, training=True)
            
            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss 
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
            #el gradient representa la direcció i la magnitud en la qual s'ha
            #d'ajustar cada paràmetre del model per reduir la pèrdua (loss) durant l'entrenament
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    '''
    Aquesta funció 'test_step' pilla la data de 'train' per avaluar-la i torna a calcular la pèrdua
    total, la pèrdua de classificació i la pèrdua de localització. Això es fa per avaluar el 
    rendiment del model en dades noves.
    '''
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [ ]:
model = Detector(detector)

El 'mode.compile' configura el model per utilitzar l'optimitzador especificat per minimitzar la combinació de les funcions de pèrdua de classificació i regressió durant el procés d'entrenament

In [ ]:
model.compile(opt, classloss, regressloss)

### 3.2 Entrenament


In [ ]:
logdir='logs' #Crea un directori on es guardarà la informació del Tensorboard 

TensorBoard és una eina de visualització interactiva que s'utilitza en l'entrenament de models de xarxes neuronals per poder entendre millor el comportament del model durant l'entrenament i ajustar els paràmetres de manera més efectiva. 


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) #crea un callback per registrar la informació del model 

In [ ]:
hist = model.fit(train, epochs=10, validation_data=val, callbacks=[tensorboard_callback])

Aquesta última línia es passa tota la informació de train (osigui crida a tota la funció d'entrenament) per 10 iteracions (epochs) i registra les dades d'entrenament i validació durant l'entrenament a partir del callback de TensorBoard, llavors "hist" conté tota la info sobre les losses (pèrdues) i l'exactitud del model durant l'entrenament

### 3.3 Anàlisi del rendiment de l'entrenament

In [ ]:
hist.history #per veure totes les pèrdues (losses)

El següent fragment mostra totes les losses de manera gràfica, en teoria tant les pèrdues i les pèrdues validades com les de classificació de localització i les totals haurien de ser valors molt semblants per a ser més exactes, en cas que no ho son deu haver-hi algun problema amb alguna anotation i no l'haurà processat bé (tampoc és un problema greu)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()